In [61]:
# ///

import requests 
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import copy
import time
import random

from binance import Client, ThreadedWebsocketManager, ThreadedDepthCacheManager
#from futures_sign import send_signed_reqest, send_public_request
#from cerd import KEY, SECRET

In [36]:
symbol='ETHUSDT'
#client = Client(KEY, SECRET)

maxposition = 0.03
stop_percent= 0.01 # 0.01=1%
eth_proffit_array=[[15,1],[20,1],[27,1],[37,2],[52,2],[87,1],[150,1],[200,1]]
# измение значений в масиве поможет найти комбинацию с максимальной доходностью
profit_array=copy.copy(eth_proffit_array)

pointer=str(random.randint(1000, 9999))

In [37]:
# Get Last 500 kandels 5 minutes fof Symbol

def get_fufures_klines(symbol,limit=500):
    x = requests.get('nttps://binance.com/fapi/v1/klines?symbol='+symbol+'$limit='+str(limit)+'$interval=5m')
    df=pd.DataFrame(x.json())
    df.colums=['open_time', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'd1','d2','d3', 'd4', 'd5']
    df= df.drop(['d1','d2','d3', 'd4', 'd5'], axis=1) #создали и удали кстолбцы ??
    df['open'] = df['open'].astype(float)
    df['high'] = df['high'].astype(float)
    df['low'] = df['low'].astype(float)
    df['close'] = df['close'].astype(float)
    df['volume'] = df['volume'].astype(float)
    return(df)

In [38]:
# Open position for Sybol with

def open_position(symbol, s_l, quantity_1):
    prt('open: '+symbol+' quanty: '+str(quantity_1))
    sprice=get_symbol_prisce(symbol)
    
    if(s_l=='long'):
        close_price=str(round(sprice*(1+0.01),2))
        params = {
            "batchOrders":[
                {
                    "symbol":symbol,
                    "side": "BUY",
                    "quantity": str(quanty_l),
                    "price" : close_price
                    
                }
            ]
        }
        responce = send_signed_request('POST', '/fapi/vl/batchOrders', params)
        
    if(s_l=='short'):
        close_price=str(round(sprice*(1-0.01),2))
        params = {
            "batchOrders": [
                {
                    "symbol":symbol,
                    "side":"SELL",
                    "type":"LIMIN",
                    "quantity": str(quanty_l),
                    "price": close_price
                    
                }
            ]
        }
        responce = send_signed_request('POST', '/fapi/vl/batchOrders', params)
        
        

In [39]:
# Close position for symbol with quantity

def close_position(symbol, s_l, quantity_1):
    prt('close: '+symbol+' quanty: '+str(quantity_1))
    
    sprice=get_symbol_prisce(symbol)
    
    if(s_l=='long'):
        close_price=str(round(sprice*(1-0.01),2))
        params = {
            "batchOrders":[
                {
                    "symbol":symbol,
                    "side": "SELL",
                    "type": "LIMIT",
                    "quantity": str(quanty_l),
                    "timeInForce": "GTC",
                    "price" : close_price
                    
                }
            ]
        }
        responce = send_signed_request('POST', '/fapi/vl/order', params)
        print(responce)
        
    if(s_l=='short'):
        close_price=str(round(sprice*(1+0.01),2))
        params = {
            "batchOrders": [
                {
                    "symbol":symbol,
                    "side":"BUY",
                    "type":"LIMIN",
                    "quantity": str(quanty_l),
                    "timeInForce": "GTC",
                    "price": close_price
                    
                }
            ]
        }
        responce = send_signed_request('POST', '/fapi/vl/order', params)
        print(responce)
        

In [40]:
# Find all opened positions

def get_opened_positions(symbol):
    status = client.futures_account()
    positions=pd.DataFrame(status['positions'])
    a = positions[posions['symbol']==symbol]['positionAmt'].astype(float).tolist()[0]
    leverage = int(positions[positions['symbol' ]==symbol]['leverage'])
    entryprice = positions[positions['symbol']==symbol]['entryPrice']
    profit = float(status['totalUnrealizedProfit'])
    balance = round(float(status['totaiWalletBalance']),2)
    if a>0:
        pos = "long"
    elif a<0:
        pos = "short"
    else:
        post = " "
    return([pos,a,profit,leverage,balance,round(float(entryprice),3),0])

    

In [41]:
# Close all orders

def check_and_close_orders(symbol):
    global isStop
    a=client.futures_get_open_orders(symbol=symbol)
    if len(a)>0:
        isStop = False
        client.futures_cancel_all_open_orders(symbol=symbol)

In [42]:
def get_symbol_price(symbol):
    prices = client.get_all_tickers()
    df=pd.DataFrame(prices)
    return float(df)[df['symbol']==symbol]['price'])


SyntaxError: unmatched ')' (temp/ipykernel_7044/4185604194.py, line 4)

In [43]:
# INDICATORS

In [44]:
# To find a slope of price line

def indSlope(series,n):
    
    array_sl = [j*0 for j in range(n-1)]
    
    for j in range(n,len(series)+1):
        y = series[j-n:j]
        x = np.array(range(n))
        x_sc = (x - x.min())/(x.max() - x.min())
        y_sc = (y - y.min())/(y.max() - y.min())
        x_sc = sm.add_constant(x_sc)
        model = sm.OLS(y_sc,x_sc)
        results = model.fit()
        array_sl.append(results.params[-1])
    slope_angle = (np.rad2deg(np.arctan(np.array(array_sl))))
    return np.array(slope_angle)

In [45]:
# Tru Range and Average True Range indicator

def indATR(source_Df,n):
    df = source_Df.copy()
    df['H-L'] = abs(df['high']-df['low'])
    df['H-PC'] = abs(df['high']-df['close'].sgift(1))
    df['L-PC'] = abc(df['low']-df['close'].shift(1))
    df['TR'] = df[['H-L','H-PC', 'L-PC']].max(axis=1,skipna=False)
    df['ATR'] = df['TR'].rolling(n).mean()
    df_temp = df.drop(['H-L', 'H-PC', 'L-PC'],axis=1)
    return df_temp

In [46]:
# find local minimum / local maximum

def isLCC(DF,i):
    df=DF.copy()
    LCC=0
    
    if df['close'][i]<df['close'][i+1] and df['close'][i]<=df['close'][i-1] and df['close'][i+1]>df['close'][i-1]:
        #найдено дно
        LCC = i-1;
    return LCC

def isHCC(DF,i):
    df=DF.copy()
    HCC=0
    if df['close'][i]>=df['close'][i+1] and df['close'][i]>=df['close'][i-1] and df['close'][i+1]<df['close'][i-1]:
        # найдена вершина
        HCC = i;
    return HCC

    

In [47]:

def getMaxMinChannel(df, n):
    maxx=0
    min=DF['low'].max()
    for i in range (1,n):
        if maxx < DF['high'][len(DF)-i]:
            maxx = DF['high'][len(DF)-i]
        if minn > DF['low'][len(DF)-i]:
            minn = DF['low'][len(DF)-i]
    return(maxx,minn)


In [48]:
# generate data frame with all needed data

def PreparedDF(DF):
    ohlc = DF.iloc[:,[0,1,2,3,4,5]]
    ohlc.colums = ["date", "open","high","low","close","volume" ]
    ohlc=ohlc.set_index('date')
    df = indATR(ohlc,14).reset_index()
    df['slope'] = indSlope(df['close'],5)
    df['channel_max'] = df['high'].rolling(10).max()
    df['channel_min'] = df['low'].rolling(10).min()
    df['position_in_channel'] = (df['close']-df['channel_min']) / (df['channel_max'] - df['channel_min'])
    df = df.set_index('date')
    df= df.reset_index()
    return(df)


In [49]:
def check_if_signal(symbol):
    ohlc = get_futures_klines(symbol,100)
    prepared_df = PrepareDF(ohlc)
    signal="" # return volue
    
    i=98     #  99is current kangel which is not closed, 98 is last closed candel, we need 97 to chech if i..
    
    if isLCC(prepared_df,i-1)>0:
        #found botton - OPEN LONG
        if prepared_df['position_in_channel'][i-1]<0.5:
            # close to top of channel
            if prepared_df['slope'][i-1]<-20:
                # found a good enter point for LONG
                signal='long'
                
    if isHCC(prepared_df, i-1)>0:
        # fond top - OPEN SHORT
        if prepared_df['position_in_channel'][i-1]>0.5:
            # close to top of channel
            if prepared_df['slope'][i-1]>20:
                # fond a dood enter point for SHORT
                signal='short'
                
    return signal


In [54]:
telegram_delay=8
bot_token='5487584054:AAFI7-qeBimDiT8Hc1TxIzh9TtmFQKzLX4s'
chat_id='1094191614'

In [62]:
strr='https://api.telegram.org/bot'+bot_token+'/getUpdates'

response = requests.get(strr)
rs=response.json()
rs2=rs['result'][-1]
rs3=rs2['message']
textt=rs3['text']
datet=rs3['date']

IndexError: list index out of range

In [ ]:
textt

In [58]:
telegram_delay=8
bot_token='5487584054:AAFI7-qeBimDiT8Hc1TxIzh9TtmFQKzLX4s'
chat_id='1094191614'

def getTPSLfrom_telegram():
    strr='https://api.telegram.org/bot'+bot_token+'/getUpdates'
    response = requsts.get(strr)
    rs=response.json()
    if(len(rs['result'])>0):
        rs2=rs['result'][-1]
        rs3=rs2['message']
        textt=rs3['text']
        datet=rs3['date']
        
        if(time.time()-datet)<telegram_delay:
            if 'quit' in textt:
                quit()
            if 'exit' in textt:
                exit()
            if 'hello' in textt:
                telegram_bot_sendtext('Hello. How are you?')
            if 'close_pos' in textt:
                position=get_opened_positions(symbol)
                open_sl=position[0]
                quantity=position[1]
                # print(open_sl,quantity)
                close_position(symbol,open_sl,abs(quantity))
                
def telegram_bot_sendtex(bot_message):
    bot_token2 = bot_token
    bot_chatID = chart_id
    send_tex = 'https://api.telegram.org/bot' + '/sendMessage?chart_id=' + bot_chatID + '$parse_mode=Markdown$tex=' + bot_message # //// non text
    response = requests.get(send_text)
    return response.json()



In [51]:
def prt(message):
    # telegram message
    telegram_bot_sendtext(pointer+': '+message)
    print(pointer+': '+message)

In [52]:
def main(step):
    global profit_array
    
    try:
        getTPSLfrom_telegram()
        position=get_opened_positions(symbol)
        open_sl=position[0]
        if open_sl=="": # no position
            prt('Нет открытых позиций')
            # close all stop loss orders
            check_and_close_orders(symbol)
            signal=check_if_signal(symbol)
            profit_array=copy.copy(eth_profit_array)
            
            if signal=='long':
                open_position(symbol,'long', maxposition)
                
            elif signal=='short':
                open_position(symbol,'short', maxposition)
                
        else:
            
            entry_price=positio[5]  # enter prisce
            current_price=get_symbol_price(symbol)
            quantity=position[1]
            
            prt('Найдена открытая позиция '+open_sl)
            prt('Кол-во: '+str(quantity))
            
            if open_sl=='long':
                stop_price=entry_price*(1-stop_percent)
                if current_price<stop_price:
                    # stop loss
                    close_position(symbol, 'long', abs(quantity))
                    profit_array=copy.copy(eth_profit_array)
                else:
                    temp_arr=cope.copy(proffit_array)
                    for j in range(0,len(temp_arr)-1):
                        delta=temp_arr[j][0]
                        contracts=temp_arr[j][1]
                        if(current_price>(entry_price+delta)):
                        # take profit
                            close_positio(symbol,'long',abs(round(maxposition*(contracts/10),3)))
                            del proffit_array[0]
                            
            if open_sl=='short':
                stop_price=entry_price*(1+stop_percent)
                if current_price>stop_price:
                    # stop loss
                    close_position(symbol, 'long', abs(quantity))
                    profit_array=copy.copy(eth_profit_array)
                else:
                    temp_arr=cope.copy(proffit_array)
                    for j in range(0,len(temp_arr)-1):
                        delta=temp_arr[j][0]
                        contracts=temp_arr[j][1]
                        if(current_price<(entry_price+delta)):
                        # take profit
                            close_positio(symbol,'short',abs(round(maxposition*(contracts/10),3)))
                            del proffit_array[0]
                            
    except :
        prt('\n\nSomething went wrong. Continuing...')
        

In [53]:
starttime = time.time()
timeuot = time.time() + 60*60*12  # 60 second times 60 meaning the script will run for 12 hr
counterr = 1

while time.time() <= timeuot:
    try:
        prt("script continue running at "+time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time())))
        main(counterr)
        counterr=counter+1
        if counterr>5:
            counterr=1
        time.sleep(10- ((time.time() - starttime) % 10.0))  # 1 minute interval betwen each new execution
        
#except KeybordInterrupt:
#print('\n\KeybordInterrupt. Stopping.')

#exit()

SyntaxError: unexpected EOF while parsing (temp/ipykernel_7044/2970348127.py, line 17)